In [1]:
import pandas as pd
import numpy as np
from os import listdir
import keras
# import cv2
from skimage.io import imread
from skimage.transform import resize

Using TensorFlow backend.


In [2]:
train_path = "/projectnb/dl-course/jxtang/EC500_proj/train/"
test_path = "/projectnb/dl-course/jxtang/EC500_proj/test/"
label_path = './train.csv'
train_files = listdir(train_path)
test_files = listdir(test_path)


In [3]:
train_labels = pd.read_csv(label_path)
train_labels.head()

,Id,Target
0,00070df0-bbc3-11e8-b2bc-ac1f6b6435d0,16 0
1,000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0,7 1 2 0
2,000a9596-bbc4-11e8-b2bc-ac1f6b6435d0,5
3,000c99ba-bba4-11e8-b2b9-ac1f6b6435d0,1
4,001838f8-bbca-11e8-b2bc-ac1f6b6435d0,18


In [4]:
names_dict = {
    0:  "Nucleoplasm",  
    1:  "Nuclear membrane",   
    2:  "Nucleoli",   
    3:  "Nucleoli fibrillar center",   
    4:  "Nuclear speckles",
    5:  "Nuclear bodies",   
    6:  "Endoplasmic reticulum",   
    7:  "Golgi apparatus",   
    8:  "Peroxisomes",   
    9:  "Endosomes",   
    10:  "Lysosomes",   
    11:  "Intermediate filaments",   
    12:  "Actin filaments",   
    13:  "Focal adhesion sites",   
    14:  "Microtubules",   
    15:  "Microtubule ends",   
    16:  "Cytokinetic bridge",   
    17:  "Mitotic spindle",   
    18:  "Microtubule organizing center",   
    19:  "Centrosome",   
    20:  "Lipid droplets",   
    21:  "Plasma membrane",   
    22:  "Cell junctions",   
    23:  "Mitochondria",   
    24:  "Aggresome",   
    25:  "Cytosol",   
    26:  "Cytoplasmic bodies",   
    27:  "Rods & rings"
}
# reverse_names_dict = dict((v,k) for k,v in names_dict.items())

In [5]:
def fill_targets(row):
    row.Target = np.array(row.Target.split(" ")).astype(np.int)
    for num in row.Target:
        name = names_dict[int(num)]
        row.loc[name] = 1
    return row

for key in names_dict.keys():
    train_labels[names_dict[key]] = 0
train_labels = train_labels.apply(fill_targets, axis=1)
train_labels["number_of_targets"] = train_labels.drop(["Id", "Target"],axis=1).sum(axis=1)

In [6]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3)
for train_idx, test_idx in kf.split(train_labels.index.values):
    partition = {}
    partition["train"] = train_labels.Id.values[train_idx]
    partition["validation"] = train_labels.Id.values[test_idx]
#     X_train, X_test = train_labels.Id.values[train_idx], train_labels.Id.values[test_idx]
#     y_train, y_test = train_labels.Id.values[train_idx], train_labels.Id.values[test_idx]

In [7]:
partition

{'train': array(['00070df0-bbc3-11e8-b2bc-ac1f6b6435d0',
        '000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0',
        '000a9596-bbc4-11e8-b2bc-ac1f6b6435d0', ...,
        'ab33dff8-bba7-11e8-b2ba-ac1f6b6435d0',
        'ab351f1c-bbb6-11e8-b2ba-ac1f6b6435d0',
        'ab385b2e-bbba-11e8-b2ba-ac1f6b6435d0'], dtype=object),
 'validation': array(['ab3978aa-bbbb-11e8-b2ba-ac1f6b6435d0',
        'ab3b9258-bbab-11e8-b2ba-ac1f6b6435d0',
        'ab3ca16e-bbc1-11e8-b2bb-ac1f6b6435d0', ...,
        'fff189d8-bbab-11e8-b2ba-ac1f6b6435d0',
        'fffdf7e0-bbc4-11e8-b2bc-ac1f6b6435d0',
        'fffe0ffe-bbc0-11e8-b2bb-ac1f6b6435d0'], dtype=object)}

In [25]:
class ModelParameters(object):
    path = train_path
    num_classes=28
    image_rows=512
    image_cols=512
    batch_size=50
    n_channels=3
    shuffle=False
    scaled_row_dim = 256
    scaled_col_dim = 256
    n_epochs=300

parameter = ModelParameters()

In [9]:
class ImagePreprocessor:
    
    def __init__(self, modelparameter):
        self.parameter = modelparameter
        self.path = self.parameter.path
        self.scaled_row_dim = self.parameter.scaled_row_dim
        self.scaled_col_dim = self.parameter.scaled_col_dim
        self.n_channels = self.parameter.n_channels
    
    def preprocess(self, image):
        image = self.resize(image)
        image = self.reshape(image)
        image = self.normalize(image)
        return image
    
    def resize(self, image):
        return resize(image, (self.scaled_row_dim, self.scaled_col_dim))
    
    def reshape(self, image):
        return np.reshape(image, (image.shape[0], image.shape[1], self.n_channels))
    
    def normalize(self, image):
#         image /= 255
#         return image
        return (image / 255.0 - 0.5) / 0.5
            
    def load_image(self, image_id):
        image = np.zeros(shape=(512,512,4))
        image[:,:,0] = imread(self.basepath + image_id + "_green" + ".png")
        image[:,:,1] = imread(self.basepath + image_id + "_blue" + ".png")
        image[:,:,2] = imread(self.basepath + image_id + "_red" + ".png")
        image[:,:,3] = imread(self.basepath + image_id + "_yellow" + ".png")
        return image[:,:,0:self.parameter.n_channels]

In [10]:
preprocessor = ImagePreprocessor(parameter)

In [11]:
class DataGenerator(keras.utils.Sequence):
    
    def __init__(self, list_IDs, labels, modelparameter, imagepreprocessor):
        self.params = modelparameter
        self.labels = labels
        self.list_IDs = list_IDs
        self.dim = (self.params.scaled_row_dim, self.params.scaled_col_dim)
        self.batch_size = self.params.batch_size
        self.n_channels = self.params.n_channels
        self.num_classes = self.params.num_classes
        self.preprocessor = imagepreprocessor
        self.shuffle = self.params.shuffle
        self.on_epoch_end()
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def get_targets_per_image(self, identifier):
        return self.labels.loc[self.labels.Id==identifier].drop(
                ["Id", "Target", "number_of_targets"], axis=1).values
            
    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size, self.num_classes), dtype=int)
        for i, identifier in enumerate(list_IDs_temp):
            image = self.preprocessor.load_image(identifier)
            image = self.preprocessor.preprocess(image)
            X[i] = image
            y[i] = self.get_targets_per_image(identifier)
        return X, y
    
    def __len__(self):
        return int(np.floor(len(self.list_IDs) / self.batch_size))
 
    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        X, y = self.__data_generation(list_IDs_temp)
        return X, y

In [12]:
class PredictGenerator:
    
    def __init__(self, predict_Ids, imagepreprocessor, predict_path):
        self.preprocessor = imagepreprocessor
        self.preprocessor.basepath = predict_path
        self.identifiers = predict_Ids
    
    def predict(self, model):
        y = np.empty(shape=(len(self.identifiers), self.preprocessor.parameter.num_classes))
        for n in range(len(self.identifiers)):
            image = self.preprocessor.load_image(self.identifiers[n])
            image = self.preprocessor.preprocess(image)
            image = image.reshape((1, *image.shape))
            y[n] = model.predict(image)
        return y

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.losses import binary_crossentropy
from keras.optimizers import Adadelta
from keras.models import load_model

class BaseLineModel:
    
    def __init__(self, modelparameter):
        self.params = modelparameter
        self.num_classes = self.params.num_classes
        self.img_rows = self.params.scaled_row_dim
        self.img_cols = self.params.scaled_col_dim
        self.n_channels = self.params.n_channels
        self.input_shape = (self.img_rows, self.img_cols, self.n_channels)
        self.my_metrics = ['accuracy']
    
    def build_model(self):
        self.model = Sequential()
        self.model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=self.input_shape))
        self.model.add(Conv2D(32, (3, 3), activation='relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Dropout(0.25))
        self.model.add(Flatten())
        self.model.add(Dense(64, activation='relu'))
        self.model.add(Dropout(0.5))
        self.model.add(Dense(self.num_classes, activation='sigmoid'))

    def compile_model(self):
        self.model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=self.my_metrics)
    
    def set_generators(self, train_generator, validation_generator):
        self.training_generator = train_generator
        self.validation_generator = validation_generator
    
    def learn(self):
        return self.model.fit_generator(generator=self.training_generator,
                    validation_data=self.validation_generator,
                    epochs=self.params.n_epochs, 
                    steps_per_epoch=100,
                    use_multiprocessing=True,
                    validation_steps=50,
                    workers=8)
    
    def score(self):
        return self.model.evaluate_generator(generator=self.validation_generator,
                                      use_multiprocessing=True, 
                                      workers=8)

    def predict(self, predict_generator):
        y = predict_generator.predict(self.model)
        return y
    
    def save(self, modeloutputpath):
        self.model.save(modeloutputpath)
    
    def load(self, modelinputpath):
        self.model = load_model(modelinputpath)

In [14]:
labels = train_labels
print("Number of samples in train: {}".format(len(partition["train"])))
print("Number of samples in validation: {}".format(len(partition["validation"])))

Number of samples in train: 20715
Number of samples in validation: 10357


In [15]:
training_generator = DataGenerator(partition['train'], labels, parameter, preprocessor)
validation_generator = DataGenerator(partition['validation'], labels, parameter, preprocessor)
predict_generator = PredictGenerator(partition['validation'], preprocessor, train_path)

In [116]:
import warnings
warnings.filterwarnings("ignore")
model = BaseLineModel(parameter)
model.build_model()
model.compile_model()
model.set_generators(training_generator, validation_generator)
history = model.learn()
#model.save("baseline_model.h5")
proba_predictions = model.predict(predict_generator)
baseline_proba_predictions = pd.DataFrame(proba_predictions, columns=train_labels.drop(
    ["Target", "number_of_targets", "Id"], axis=1).columns)
baseline_proba_predictions.to_csv("baseline_predictions.csv")

Epoch 1/1


Process ForkPoolWorker-124:
Process ForkPoolWorker-117:
Process ForkPoolWorker-120:
Process ForkPoolWorker-123:
Process ForkPoolWorker-118:
Traceback (most recent call last):
Process ForkPoolWorker-119:
  File "/share/pkg/python/3.6.2/install/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/share/pkg/python/3.6.2/install/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/share/pkg/python/3.6.2/install/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/share/pkg/python/3.6.2/install/lib/python3.6/site-packages/keras/utils/data_utils.py", line 371, in get_index
    return ds[i]
  File "<ipython-input-111-f1c9b30f20f0>", line 40, in __getitem__
    X, y = self.__data_generation(list_IDs_temp)
  File "<ipython-input-111-f1c9b30f20f0>", line 28, in __data_generation
    image = self.preprocessor.load_image(identifier)
  File "<ip

  File "<ipython-input-109-20bfa20f1de1>", line 17, in resize
    return resize(image, (self.scaled_row_dim, self.scaled_col_dim))
  File "<ipython-input-109-20bfa20f1de1>", line 17, in resize
    return resize(image, (self.scaled_row_dim, self.scaled_col_dim))
KeyboardInterrupt
  File "/share/pkg/python/3.6.2/install/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/share/pkg/python/3.6.2/install/lib/python3.6/site-packages/skimage/transform/_warps.py", line 135, in resize
    preserve_range=preserve_range)
  File "/share/pkg/python/3.6.2/install/lib/python3.6/site-packages/skimage/transform/_warps.py", line 135, in resize
    preserve_range=preserve_range)
  File "/share/pkg/python/3.6.2/install/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/share/pkg/python/3.6.2/install/lib/python3.6/site-packages/skimage/transform/_warps.py", line 819, in warp
    _clip_wa

  File "/share/pkg/python/3.6.2/install/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/share/pkg/python/3.6.2/install/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/share/pkg/python/3.6.2/install/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/share/pkg/python/3.6.2/install/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/share/pkg/python/3.6.2/install/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/share/pkg/python/3.6.2/install/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/share/pkg/python/3.6.2/install/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/share/pkg/python/3.6.2/install/lib/python3.6/multiprocessing

KeyboardInterrupt: 

Process ForkPoolWorker-146:
Process ForkPoolWorker-137:
Process ForkPoolWorker-144:
Process ForkPoolWorker-143:
Process ForkPoolWorker-138:
Process ForkPoolWorker-139:
Process ForkPoolWorker-145:
Process ForkPoolWorker-142:
Process ForkPoolWorker-141:
Process ForkPoolWorker-140:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/share/pkg/python/3.6.2/install/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/share/pkg/python/3.6.2/install/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/share/pkg/python/3.6.2/install/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  Fi

In [30]:
baseline_proba_predictions

,Nucleoplasm,Nuclear membrane,Nucleoli,Nucleoli fibrillar center,Nuclear speckles,Nuclear bodies,Endoplasmic reticulum,Golgi apparatus,Peroxisomes,Endosomes,...,Microtubule organizing center,Centrosome,Lipid droplets,Plasma membrane,Cell junctions,Mitochondria,Aggresome,Cytosol,Cytoplasmic bodies,Rods & rings
0,0.376832,0.046780,0.061799,0.086671,0.022863,0.048857,0.143351,0.094772,0.017296,0.049911,...,0.031554,0.043320,0.073627,0.077283,0.047587,0.201696,0.029957,0.294167,0.037141,0.030636
1,0.379009,0.055425,0.074850,0.100301,0.029612,0.062742,0.159723,0.110091,0.024523,0.063089,...,0.039895,0.051550,0.084751,0.087392,0.058937,0.214846,0.038274,0.310294,0.045202,0.039707
2,0.350029,0.050695,0.064352,0.094351,0.025668,0.054158,0.139662,0.116092,0.022074,0.056161,...,0.039601,0.047033,0.074418,0.072260,0.050324,0.181006,0.033442,0.247697,0.037618,0.032566
3,0.345599,0.053750,0.067764,0.101102,0.027440,0.056822,0.142196,0.125807,0.024467,0.059165,...,0.043356,0.049655,0.077290,0.073665,0.052880,0.180115,0.036528,0.237169,0.039492,0.034519
4,0.393301,0.072534,0.091997,0.120062,0.040748,0.075797,0.180900,0.130547,0.031854,0.076789,...,0.051189,0.066905,0.102592,0.108494,0.070740,0.226998,0.048502,0.313710,0.058137,0.049274
5,0.364394,0.056697,0.072970,0.101948,0.030591,0.061433,0.151446,0.117273,0.025211,0.063906,...,0.042773,0.053786,0.080611,0.084409,0.056152,0.192301,0.037529,0.269018,0.044169,0.037816
6,0.353359,0.053774,0.068572,0.101403,0.028175,0.058123,0.144633,0.120700,0.024471,0.060497,...,0.042393,0.050409,0.078038,0.075983,0.053265,0.185011,0.036509,0.250154,0.040464,0.035083
7,0.364654,0.054069,0.070941,0.095332,0.028438,0.058604,0.150786,0.110215,0.022985,0.060748,...,0.040020,0.050830,0.080434,0.083146,0.054772,0.194876,0.035535,0.277231,0.042854,0.036196
8,0.357120,0.058195,0.073742,0.103477,0.030490,0.061876,0.152743,0.125696,0.026007,0.063538,...,0.045736,0.053361,0.083630,0.081620,0.057115,0.193401,0.039230,0.261425,0.044124,0.037734
9,0.389428,0.069030,0.088897,0.114983,0.040422,0.076736,0.177646,0.129077,0.031054,0.076533,...,0.052691,0.065328,0.099702,0.108332,0.068749,0.228085,0.048842,0.320969,0.059272,0.049704


In [16]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12509767357886614562
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11273211085
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17876244925859581285
physical_device_desc: "device: 0, name: Tesla K40m, pci bus id: 0000:04:00.0, compute capability: 3.5"
]


In [17]:
import keras.backend as K
import tensorflow as tf

def base_f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return f1

def f1_min(y_true, y_pred):
    f1 = base_f1(y_true, y_pred)
    return K.min(f1)

def f1_max(y_true, y_pred):
    f1 = base_f1(y_true, y_pred)
    return K.max(f1)

def f1_mean(y_true, y_pred):
    f1 = base_f1(y_true, y_pred)
    return K.mean(f1)

def f1_std(y_true, y_pred):
    f1 = base_f1(y_true, y_pred)
    return K.std(f1)

In [18]:
class TrackHistory(keras.callbacks.Callback):
    
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

In [19]:
class ImprovedDataGenerator(DataGenerator):
    
    # in contrast to the base DataGenerator we add a target wishlist to init
    def __init__(self, list_IDs, labels, modelparameter, imagepreprocessor, target_wishlist):
        super().__init__(list_IDs, labels, modelparameter, imagepreprocessor)
        self.target_wishlist = target_wishlist
    
    def get_targets_per_image(self, identifier):
        return self.labels.loc[self.labels.Id==identifier][self.target_wishlist].values


In [22]:
use_dropout = False
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.layers import BatchNormalization
class ImprovedModel(BaseLineModel):
    
    def __init__(self, modelparameter,
                 use_dropout,
                 my_metrics=[f1_mean, f1_std, f1_min, f1_max]):
        
        super().__init__(modelparameter)
        self.my_metrics = my_metrics
        self.use_dropout = use_dropout
        
    def learn(self):
        self.history = TrackHistory()
        return self.model.fit_generator(generator=self.training_generator,
                    validation_data=self.validation_generator,
                    epochs=self.params.n_epochs, 
                    use_multiprocessing=True,
                    workers=8,
                    steps_per_epoch=100,
                    validation_steps=50,                                     
                    callbacks = [self.history])
    
    def build_model(self):
        self.model = Sequential()

        # Block 1
        self.model.add(Conv2D(64, (3, 3), padding='same', name='conv1_1', input_shape = self.input_shape))
        self.model.add(BatchNormalization(name='bn1_1'))
        self.model.add(Activation('relu', name='relu1_1'))
        self.model.add(Conv2D(64, (3, 3), padding='same', name='conv1_2'))
        self.model.add(BatchNormalization(name='bn1_2'))
        self.model.add(Activation('relu', name='relu1_2'))
        self.model.add(MaxPooling2D((2, 2), strides=(2, 2), name='pool1'))

        # Block 2
        self.model.add(Conv2D(128, (3, 3), padding='same', name='conv2_1'))
        self.model.add(BatchNormalization(name='bn2_1'))
        self.model.add(Activation('relu', name='relu2_1'))
        self.model.add(Conv2D(128, (3, 3), padding='same', name='conv2_2'))
        self.model.add(BatchNormalization(name='bn2_2'))
        self.model.add(Activation('relu', name='relu2_2'))
        self.model.add(MaxPooling2D((2, 2), strides=(2, 2), name='pool2'))

        # Block 3
        self.model.add(Conv2D(256, (3, 3), padding='same', name='conv3_1'))
        self.model.add(BatchNormalization(name='bn3_1'))
        self.model.add(Activation('relu', name='relu3_1'))
        self.model.add(Conv2D(256, (3, 3), padding='same', name='conv3_2'))
        self.model.add(BatchNormalization(name='bn3_2'))
        self.model.add(Activation('relu', name='relu3_2'))
        self.model.add(Conv2D(256, (3, 3), padding='same', name='conv3_3'))
        self.model.add(BatchNormalization(name='bn3_3'))
        self.model.add(Activation('relu', name='relu3_3'))
        self.model.add(Conv2D(256, (3, 3), padding='same', name='conv3_4'))
        self.model.add(BatchNormalization(name='bn3_4'))
        self.model.add(Activation('relu', name='relu3_4'))
        self.model.add(MaxPooling2D((2, 2), strides=(2, 2), name='pool3'))

        # Classification block
        self.model.add(Flatten(name='flatten'))
        self.model.add(Dense(512, name='ip1'))
        self.model.add(BatchNormalization(name='bn4'))
        self.model.add(Activation('relu', name='relu4'))
        self.model.add(Dropout(0.5))
        self.model.add(Dense(512, name='ip2'))
        self.model.add(BatchNormalization(name='bn5'))
        self.model.add(Activation('relu', name='relu5'))
        self.model.add(Dense(28, activation='softmax'))

In [23]:
parameter = ModelParameters()
preprocessor = ImagePreprocessor(parameter)
labels = train_labels
training_generator = DataGenerator(partition['train'], labels,
                                           parameter, preprocessor)
validation_generator = DataGenerator(partition['validation'], labels,
                                             parameter, preprocessor)
predict_generator = PredictGenerator(partition['validation'], preprocessor, train_path)

In [ ]:
model = ImprovedModel(parameter, use_dropout=use_dropout)
model.build_model()
model.compile_model()
model.set_generators(training_generator, validation_generator)
epoch_history = model.learn()
proba_predictions = model.predict(predict_generator)
#model.save("improved_model.h5")
improved_proba_predictions = pd.DataFrame(proba_predictions)

Epoch 1/300
100/100 [==============================] - 87s - loss: 0.1902 - f1_mean: 0.0118 - f1_std: 0.0490 - f1_min: 0.0000e+00 - f1_max: 0.2416 - val_loss: 0.1853 - val_f1_mean: 0.0000e+00 - val_f1_std: 0.0000e+00 - val_f1_min: 0.0000e+00 - val_f1_max: 0.0000e+00
Epoch 2/300
100/100 [==============================] - 68s - loss: 0.1769 - f1_mean: 0.0143 - f1_std: 0.0639 - f1_min: 0.0000e+00 - f1_max: 0.3248 - val_loss: 0.1852 - val_f1_mean: 0.0000e+00 - val_f1_std: 0.0000e+00 - val_f1_min: 0.0000e+00 - val_f1_max: 0.0000e+00
Epoch 3/300
100/100 [==============================] - 67s - loss: 0.1721 - f1_mean: 0.0145 - f1_std: 0.0650 - f1_min: 0.0000e+00 - f1_max: 0.3323 - val_loss: 0.1815 - val_f1_mean: 7.0683e-04 - val_f1_std: 0.0037 - val_f1_min: 0.0000e+00 - val_f1_max: 0.0198
Epoch 4/300
100/100 [==============================] - 66s - loss: 0.1701 - f1_mean: 0.0150 - f1_std: 0.0677 - f1_min: 0.0000e+00 - f1_max: 0.3463 - val_loss: 0.1799 - val_f1_mean: 0.0106 - val_f1_std: 0.055

100/100 [==============================] - 68s - loss: 0.0524 - f1_mean: 0.4255 - f1_std: 0.4185 - f1_min: 0.0000e+00 - f1_max: 1.0000 - val_loss: 0.1621 - val_f1_mean: 0.1539 - val_f1_std: 0.2656 - val_f1_min: 0.0000e+00 - val_f1_max: 0.8728
Epoch 65/300
100/100 [==============================] - 70s - loss: 0.0525 - f1_mean: 0.4321 - f1_std: 0.4168 - f1_min: 0.0000e+00 - f1_max: 1.0000 - val_loss: 0.1966 - val_f1_mean: 0.1173 - val_f1_std: 0.2247 - val_f1_min: 0.0000e+00 - val_f1_max: 0.7642
Epoch 66/300
100/100 [==============================] - 68s - loss: 0.0528 - f1_mean: 0.4300 - f1_std: 0.4143 - f1_min: 0.0000e+00 - f1_max: 1.0000 - val_loss: 0.1807 - val_f1_mean: 0.1214 - val_f1_std: 0.2282 - val_f1_min: 0.0000e+00 - val_f1_max: 0.7730
Epoch 67/300
100/100 [==============================] - 69s - loss: 0.0532 - f1_mean: 0.4353 - f1_std: 0.4176 - f1_min: 0.0000e+00 - f1_max: 1.0000 - val_loss: 0.1691 - val_f1_mean: 0.1226 - val_f1_std: 0.2310 - val_f1_min: 0.0000e+00 - val_f1_m